Note: Pytorch模型训练实用教程（余霆嵩）
----
<https://github.com/tensor-yu/PyTorch_Tutorial>
Chapter4: 监控模型——可视化
---

**4.3 特征图可视化**    
1. 获取图片，将其转换成图片输入前的数据格式，即一系列transform,   
2. 获取模型各层操作，手动的执行每一层操作，拿到所需的feature maps,   
3. 借助tensorboardX进行绘制   
tips: 此处获取模型各层操作是__init__()中定义的操作，然而模型真实运行采用的是forward(),所以需要人工比对两者差异。本例的差异是，__init__()中缺少激活函数relu.  

In [3]:
import os
import torch
import torchvision.utils as vutils
import numpy as np
from tensorboardX import SummaryWriter
import torch.nn.functional as F
import torchvision.transforms as transforms
import sys
sys.path.append("..")
from utils.utils import MyDataset, Net, normalize_invert
from torch.utils.data import DataLoader  

In [30]:
vis_layer = 'conv1'
log_dir = os.path.join("Result", "visual_featuremaps")
txt_path = os.path.join("Data", "train.txt") #读取数据的文本
pretrained_path = os.path.join("YuT","2_model","net_params.pkl")

net= Net()
pretrained_dict = torch.load(pretrained_path)
net.load_state_dict(pretrained_dict)
print(txt_path)

Data\train.txt


In [31]:
# 数据预处理
normMean = [0.49139968, 0.48215827, 0.44653124]
normStd = [0.24703233, 0.24348505, 0.26158768]
normTransform = transforms.Normalize(normMean, normStd)
testTransform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    normTransform
])
# 载入数据
test_data = MyDataset(txt_path=txt_path, transform=testTransform)
test_loader = DataLoader(dataset = test_data, batch_size=1) # 划分了batchsize=1
img, label = iter(test_loader).next() # 只取出了一个batch， 也就是一张图片。

In [37]:
x = img
writer = SummaryWriter(log_dir=log_dir)
for name, layer in net._modules.items():
    # 为fc层预处理x
    x = x.view(x.size(0),-1) if 'fc' in name else x
    #对x执行单层运算
    x = layer(x)
    print(x.size())
    # 由于__init()相较于forward()缺少relu操作，需要手动增加
    x = F.relu(x) if 'conv' in name else x
    # 依据选择的层，进行记录feature maps
    if name == vis_layer:
        # 绘制feature maps
        x1 = x.transpose(0,1) # C,B,H,W ---> B, C,H,W
        img_grid = vutils.make_grid(x1, normalize = True, scale_each = True, nrow=2)
        writer.add_image(vis_layer + '_feature_maps', img_grid, global_step=666)
        
        #绘制原始图像
        img_raw = normalize_invert(img, normMean, normStd) # 图像标准化
       # img_raw = np.array(img_raw *255).clip(0,255).squeeze().astype('uint8') # 这是灰色的
        img_raw = np.array(img_raw *255).squeeze() # 正常的颜色了
        writer.add_image('raw img', img_raw, global_step=666) 
writer.close()

torch.Size([1, 6, 28, 28])
torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 10, 10])
torch.Size([1, 16, 5, 5])
torch.Size([1, 120])
torch.Size([1, 84])
torch.Size([1, 10])
